<a href="https://colab.research.google.com/github/buikhanhhung/Detection-and-tracking-object-with-natural-language-processing-in-video/blob/main/Copy_of_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install numpy
!pip install torch
!pip install torchvision
!pip install transformers
!pip install Pillow
!pip install ultralytics
!pip install ipywidgets
!pip install google-colab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import cv2  # Thư viện OpenCV để xử lý video và ảnh.
import numpy as np  # Thư viện NumPy để xử lý dữ liệu số học.
from transformers import CLIPProcessor, CLIPModel  # Thư viện Transformers để sử dụng mô hình CLIP.
from PIL import Image  # Thư viện PIL để xử lý ảnh.
import torch  # Thư viện PyTorch để sử dụng GPU và các mô hình AI.
from ultralytics import YOLO  # Thư viện YOLO để phát hiện đối tượng.
from google.colab import files  # Công cụ hỗ trợ tải file lên/xuống trong Google Colab.
import os  # Thư viện xử lý các tác vụ liên quan đến hệ thống tệp.


"""
Phương pháp phát hiện và theo dõi đối tượng:
Bước 1: Lấy từng khung hình trong video
Bước 2: Phát hiện các đối tượng trong khung hình bằng YOLOv8
Bước 3: Kết hợp các đối tượng đã phát hiện ở bước 2 với mô tả ngôn ngữ tự nhiên bằng CLIP
Bước 4: Chọn đối tượng có độ tương thích cao nhất
Bước 5: Hiển thị bounding box chứa đối tượng lên video
Bước 6: Lăp lại 5 bước trên qua mỗi khung hình để theo dõi đối tượng ở khung hình tiếp theo
"""




# Khởi tạo YOLO và CLIP
device = "cuda" if torch.cuda.is_available() else "cpu"  # Sử dụng GPU nếu khả dụng, nếu không thì sử dụng CPU.
yolo = YOLO("yolov8s.pt")  # Tải mô hình YOLOv8 pre-trained từ file "yolov8s.pt".
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)  # Tải mô hình CLIP pre-trained.
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")  # Bộ xử lý dữ liệu cho CLIP.

# Hàm phát hiện và theo dõi đối tượng
def detect_and_track(video_path, text_description, output_path="output_video.mp4", similarity_threshold=0.2):
    """
    Goal: Phát hiện và theo dõi đối tượng trong video, so sánh với mô tả văn bản,
          vẽ bounding box lên đối tượng nếu độ tương đồng lớn hơn ngưỡng.

    Input:
    - video_path (str): Đường dẫn đến video đầu vào.
    - text_description (str): Mô tả văn bản của đối tượng cần phát hiện trong video.
    - output_path (str): Đường dẫn để lưu video đầu ra (mặc định "output_video.mp4").
    - similarity_threshold (float): Ngưỡng độ tương đồng cosine giữa đối tượng và mô tả (mặc định 0.2).

    Output:
    - Video đầu ra sẽ được lưu tại output_path, với các đối tượng có độ tương đồng cao với mô tả được đánh dấu bằng bounding box.
    - Video đã được xử lý sẽ tự động tải xuống máy tính của người dùng.
    """
    try:
        cap = cv2.VideoCapture(video_path)  # Mở video từ đường dẫn.
        if not cap.isOpened():  # Kiểm tra nếu không mở được video.
            raise Exception("Không thể mở video. Kiểm tra đường dẫn video.")

        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Lấy chiều rộng của khung hình video.
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Lấy chiều cao của khung hình video.
        fps = int(cap.get(cv2.CAP_PROP_FPS))  # Lấy số khung hình mỗi giây (FPS).
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
        # Khởi tạo đối tượng ghi video đầu ra.

        frame_index = 0  # Biến đếm số khung hình đã xử lý.
        while cap.isOpened():  # Lặp qua các khung hình của video.
            """
            Bước 1: Lấy từng khung hình trong video
            """
            ret, frame = cap.read()  # Đọc một khung hình từ video.
            if not ret:  # Dừng vòng lặp nếu không còn khung hình.
                break
            if frame_index % 3 != 0:  # Chỉ xử lý mỗi khung hình thứ 3 để giảm tải xử lý.
                frame_index += 1
                continue
            frame_index += 1
            """
            Bước 2: Phát hiện các đối tượng trong khung hình bằng YOLOv8
            """
            frame = cv2.convertScaleAbs(frame, alpha=1.5, beta=20)
            # Tăng độ sáng và độ tương phản của khung hình.
            pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            # Chuyển khung hình từ định dạng OpenCV (BGR) sang định dạng PIL (RGB).
            results = yolo(frame)  # Phát hiện đối tượng trong khung hình bằng YOLO.
            if len(results[0].boxes) == 0:  # Nếu không phát hiện được đối tượng, bỏ qua khung hình này.
                continue

            boxes = results[0].boxes.xyxy.cpu().numpy()  # Lấy toạ độ các bounding box dưới dạng mảng NumPy.
            scores = results[0].boxes.conf.cpu().numpy()  # Lấy các giá trị độ tin cậy (confidence scores).
            """
            Bước 3: Kết hợp các đối tượng đã phát hiện ở bước 2 với mô tả ngôn ngữ tự nhiên bằng CLIP
            """
            # Tính đặc trưng của mô tả văn bản bằng CLIP.
            inputs_text = clip_processor(text=[text_description], images=None, return_tensors="pt", padding=True).to(device)
            text_features = clip_model.get_text_features(**inputs_text)  # Đặc trưng vector của văn bản.

            max_similarity = -1  # Biến lưu độ tương đồng lớn nhất giữa đối tượng và mô tả văn bản.
            best_box = None  # Biến lưu bounding box tốt nhất.
            for i, box in enumerate(boxes):  # Lặp qua các bounding box được phát hiện.
                if scores[i] < 0.5:  # Bỏ qua các bounding box có độ tin cậy nhỏ hơn 0.5.
                    continue
                x1, y1, x2, y2 = map(int, box)  # Chuyển đổi toạ độ bounding box sang số nguyên.
                cropped_img = pil_image.crop((x1, y1, x2, y2))  # Cắt ảnh đối tượng từ khung hình.
                inputs_image = clip_processor(images=cropped_img, text=None, return_tensors="pt").to(device)
                # Tính đặc trưng vector của ảnh đối tượng.
                image_features = clip_model.get_image_features(**inputs_image)
                similarity = torch.cosine_similarity(text_features, image_features).item()
                # Tính độ tương đồng cosine giữa văn bản và ảnh.
                """
                  Bước 4: Chọn đối tượng có độ tương thích cao nhất
                """
                if similarity > max_similarity and similarity > similarity_threshold:
                    # Cập nhật bounding box tốt nhất nếu độ tương đồng lớn hơn ngưỡng.
                    max_similarity = similarity
                    best_box = (x1, y1, x2, y2)
                """
                Bước 5: Hiển thị bounding box chứa đối tượng lên video
                """
            if best_box:  # Nếu tìm thấy đối tượng phù hợp.
                x1, y1, x2, y2 = best_box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # Vẽ bounding box lên khung hình.
                cv2.putText(frame, f"{text_description} ({max_similarity:.2f})", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                # Hiển thị mô tả đối tượng và độ tương đồng lên khung hình.

            out.write(frame)  # Ghi khung hình đã xử lý vào file video đầu ra.

        cap.release()  # Đóng video đầu vào.
        out.release()  # Đóng video đầu ra.
        print(f"Video đã được lưu tại {output_path}")

        # Tự động tải file xuống máy tính.
        files.download(output_path)

    except Exception as e:  # Bắt lỗi trong quá trình xử lý.
        print(f"Lỗi: {e}")

# Hàm tải file video lên
def upload_video():
    """
    Goal: Cung cấp giao diện để người dùng tải video lên Google Colab.

    Input:
    - Không có tham số đầu vào.

    Output:
    - file_path (str): Đường dẫn tới file video đã tải lên trong môi trường Google Colab.
    """
    print("Vui lòng tải lên một video.")
    uploaded = files.upload()  # Hiển thị giao diện tải file lên trong Google Colab.

    # Lấy file đầu tiên trong danh sách file đã tải.
    for filename in uploaded.keys():
        file_path = f"/content/{filename}"  # Đường dẫn lưu file trong môi trường Colab.
        with open(file_path, "wb") as f:  # Lưu nội dung file.
            f.write(uploaded[filename])
        print(f"File {filename} đã được lưu tại: {file_path}")
        return file_path  # Trả về đường dẫn file.

# Giao diện đầu vào từ người dùng
def create_interface():
    """
    Goal: Cung cấp giao diện người dùng để tải video lên, nhập mô tả đối tượng và gọi hàm xử lý video.

    Input:
    - Không có tham số đầu vào.

    Output:
    - Gọi hàm detect_and_track để xử lý video với mô tả đối tượng đã nhập.
    """
    video_path = upload_video()  # Yêu cầu người dùng tải video lên.
    text_description = input("Nhập mô tả đối tượng ")  # Yêu cầu người dùng nhập mô tả đối tượng.
    output_path = "output_video.mp4"  # Đường dẫn lưu video đầu ra.

    if not text_description.strip():  # Kiểm tra nếu người dùng không nhập mô tả.
        print("Vui lòng nhập mô tả đối tượng.")
    else:
        detect_and_track(video_path, text_description, output_path)  # Gọi hàm xử lý.

# Chạy giao diện
create_interface()  # Bắt đầu chương trình.


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 83.2MB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Vui lòng tải lên một video.


Saving people.webm to people.webm
File people.webm đã được lưu tại: /content/people.webm

0: 384x640 20 persons, 1 car, 1 motorcycle, 436.3ms
Speed: 8.4ms preprocess, 436.3ms inference, 21.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 2 cars, 1 motorcycle, 330.0ms
Speed: 3.7ms preprocess, 330.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 1 car, 1 motorcycle, 612.9ms
Speed: 3.5ms preprocess, 612.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 persons, 1 car, 1 motorcycle, 334.5ms
Speed: 3.8ms preprocess, 334.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 persons, 1 bicycle, 2 cars, 1 motorcycle, 327.0ms
Speed: 3.4ms preprocess, 327.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 persons, 1 bicycle, 2 cars, 2 motorcycles, 312.5ms
Speed: 3.1ms preprocess, 312.5ms inference, 1.3ms postprocess per image at s

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>